<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335206/logos/Logo_des_Forschungszentrums_J_C3_BClich_seit_2018_hcliq4.svg"  width=250 align='left' style="margin-top:30px"/>
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637333672/logos/ebrains_logo_swnlzv.png" width="280" align='left' style="margin-left:50px; margin-top:25px">
</div> 

<br><br><br><br>
<br>

# Simulation of dose-response curves of antagonists using affinity values

In this tutorial we will simulate a mathematical model of a signaling pathway to obtain dose-response curves of antagonists, from wich their *potencies (IC$_{50}$)* can be infered. 


<div style='padding:15px'>
<a href="https://colab.research.google.com/github/rribeiro-sci/SSBtoolkit/blob/main/SSBtoolkit-Tutorial2.ipynb" target="_blank">
<img alt="Colab" src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335713/badges/colab-badge_hh0uyl.svg" height="25" style="margin:20px">
</a>
</div>  


## Installing the SSB toolkit library

In [ ]:
!pip install ssbtoolkit

### Importing sependencies

In [ ]:
import ssbtoolkit as ssb
import numpy as np
import pandas as pd

## Loading experimental data

Once the SSB environment is set up we are ready to start to simulate.

We will begin by loading the affinity data of the Adenosine 2A receptor natural ligand (the adenosine) as well as   of some antagonists. The experimental values were taken from *[Guide to Pharmacology](https://www.guidetopharmacology.org/GRAC/ObjectDisplayForward?objectId=19&familyId=3&familyType=GPCR) website*. This data can be downloaded directly from the repository.

In [ ]:
!wget https://raw.githubusercontent.com/rribeiro-sci/SSBtoolkit/main/example/A2AR_antagonists_data.csv

In [ ]:
data = pd.read_csv('2AR_antagonists_data.csv')
data

In <b>Tutorial1</b> the affinity values were directly used to calculate the fraction of occupied receptors according to the occupancy theory. However, such protocol is just valid if we are dealing with agonists. In this tutorial, instead, we intend to simulate a competitive antagonism. By definition, a competitive antagonist is a drug that inhibits the action of an agonist, having no effect in the absence of the agonist.

Therefore, applying the following equation, we can calculate the fraction of occupied receptors by the agonists in the presence of an antaogist:

$\frac{[L_1R]}{{R_{total}}}=\frac{[L_1]}{[L_1]+K_{d_{L1}}\big(1+\frac{[L_2]}{K_{d_{L2}}}\big)}$

Experimentally, studies of antagonists are normally conducted through radioligand binding assays. In this kind of assays, the receptor is saturated first with the agonist, and then, aliquotes of antagonist are applied to the system in order to reproduce an inhibition curve. 

To reproduce this experiment *in silico* we have to begin with the calculation of the concentration value of the radioligand/agonist (in our case the adenosine) that saturates the receptor. To achieve this, we will use the SSBtoolkit built-in function `Binding.Bind()` to obtain a ligand-receptor binding curve. After we will use the `SubMaxConcentration()`method to interpolate the submaximal (saturate) concentration value.

📖 See [The SSB toolkit](https://github.com/rribeiro-sci/SSBtoolkit/blob/main/docs/ssb_toolkit.md) documentation for more details.

In [ ]:
# Setting the ligand concentration range
lig_conc_min = 1E-4 # μM
lig_conc_max = 1E2 # μM
lig_conc_range = np.geomspace(lig_conc_min, lig_conc_max, 20)

# Setting receptor concentration
receptor_conc = 1E-3 #μM

# Getting adenosine pKi from experimental data
adenosine_pKi = data[data.Compound == 'adenosine'].pKi.item()

In [ ]:
# To obtain the ligand-receptor binding curve for adenosine we will start by initiating a binding instance
adenosine_binding = ssb.Binding()

# Performing the binding calculation
adenosine_binding.Bind(receptor_conc=receptor_conc, lig_conc_range=lig_conc_range, pKd=adenosine_pKi)

#Extraction of the submaximal concentration value
adenosine_binding.SubMaxConcentration()

#Plot of the ligand-receptor binding curve
adenosine_binding.ShowCurve()

The submaximal concentration value can be accessed by the `submax_concentration` attribute

In [ ]:
submax = adenosine_binding.submax_concentration
submax

Once we have the submaximal concentration value of adenosine that saturates the receptor we can calculate the fraction of receptors occupied by the adenosine in the presence of a range of concentration of antagonists. This calculation is automatically done during the simulation.

## Selection of  the signaling pathway 

The core of the SSB toolkit is, naturally, the mathematical models of the signaling pathways. 

Since G-protein sub-families are classified by their $\alpha$ subunits, this classfication as been served to identify their signaling pathways:
* G<sub>s</sub>
* G<sub>i/o</sub>
* G<sub>q/11</sub>
* G<sub>12/13</sub>

📖 See [The SSB toolkit](https://github.com/rribeiro-sci/SSBtoolkit/blob/main/docs/ssb_toolkit.md) documentation for more details.

We can define manualy the G$\alpha$ pathway we want to work with, or simply query our internal database of human GPCR pathways using the UNIPROT id of our receptor. The UNIPROT id for the human Adenosine A2 receptot is [P29274](https://www.uniprot.org/uniprot/P29274).

In [ ]:
uniprotID = 'P29274'
gprotein = ssb.Utils.GetGProtein(uniprotID)
gprotein

<span style="color:black">⚠️ WARNING: our toolkit was specifically design for human GPCRs. The quering for pathways for GPCRs other than Human may be included in the future. However you want to study a non-human GPCR you can still use the SSB toolkit by setting manually the G$\alpha$ pathway.</span>

##  Preparation, Simulation and Analysis

To obtain a dose-response curve from the simulation of signaling pathways, individual simulations of the pathway according to a series of ligand concentrations must be performed (as it would be done in the wet-lab).  

To define an array of ligand concentrations we will use a geometric progression. The reason why we use a geometric progression is due to the impact of the dilution fraction on the accuracy of K<sub>d</sub> and EC<sub>50</sub>/IC<sub>50</sub> values experimentally estimated. This factor, that defines the spacing between the adjacent concentration values, has an impact on the concentration values that are on the linear portion of the curve. Therefore, using a geometric progression we can mimic the experimental conditions where each concentration equals to the power of 2 of the previous lowest concentration *([Sebaugh, J.L., 2011](https://doi.org/10.1002/pst.426))*

<span style="color:black"> ⚠️ WARNING: the SSB toolkit uses μM as default concentration units.</span>


In [ ]:
# the range of ligand concentration
lig_conc_min = 1E-4 # μM
lig_conc_max = 1E4  # μM
lig_conc_range = np.geomspace(lig_conc_min, lig_conc_max, 20) # 20 concentration values

# Setting receptor concentration
receptor_conc = 1E-3 #μM

# defining other simulation parameters:
time = 10000  # time of simulation in seconds
nsteps = 1000   # number of time steps

## Integration of ODEs 

After having defined all the simulation parameters we are ready to proceed with the simulations. A simulation of a methamatical model of a signaling pathway consists of the integration of a set of ordinary differential equations (ODEs) as function of time. Since each ODE represents a molecular event of the signaling pathway, when integrated over time, such equations can describe how the concentration of species inside the model changes over time. The key point of this tutorial is the use of the drug-receptor affinity value (K<sub>d</sub>) to fire up the model. With the K<sub>d</sub> values one can calculate the fraction of receptors that are occupied by the ligand in the equilibrium and, according to the *occupancy theory*, the fraction of occupied receptors represents the concentration of activated receptors in the equilibrium *([Kenakin T., 2004 ](https://doi.org/10.1016/j.tips.2004.02.012))*. 📖 Read the [Docs](https://ssbtoolkit.readthedocs.io/) for more details.


In this tutorial we want to simulate dose-response curves of antaoginsts towards Adenosine A2 receptor, so, we will use the SSBtoolkit built-in class `Simulation.Inhibition()`. 


<span style='color:black'>ℹ️ If you want study agonist only follow the tutorial [Simulation of dose-responses curves from affinity values](https://github.com/rribeiro-sci/SSBtoolkit/blob/main/SSBtoolkit-Tutorial1.ipynb).</span>


In [ ]:
# we will start by creating a simulation instance.
sim = ssb.Simulation.Inhibition()

In [ ]:
# configuration of simulation parameters
sim.SetSimulationParameters(agonist='adenosine',
                            agonist_affinity=adenosine_pKi,
                            agonist_submaximal_conc=submax,
                            antagonists=data.Compound.to_list()[1:3],
                            antagonists_affinities=data.pKi.to_list()[1:3], 
                            lig_conc_range=lig_conc_range, 
                            receptor_conc=receptor_conc, 
                            pathway=gprotein, 
                            ttotal=time, 
                            nsteps=nsteps, 
                            binding_kinetics=False, 
                            binding_kinetic_parameters=None)

In [ ]:
#Start the simulation
sim.Run()

In the end, the concentration values of the species of the signaling pathway over the simulation time will be saved inside the instance.

The response of a signaling pathway is, naturally, represented by the increase or decrease of one of the species described by the model. So, to predict the dose-response curve we need, firstly, to extract the maximum concentration value orbserved for one specie from each individual simulation (from the series of simulations for each ligand concentration). Then, such values will be fitted to a logistic regression. 

To achieve this, we will use the analysis function:

In [ ]:
sim.Analysis()

We can now to plot the dose-response curves:

In [ ]:
sim.ShowCurve()

Finnaly, from the dose-response curves we can interpolate the IC<sub>50</sub> values.


In [ ]:
sim.ShowPotency()

💡 The potency predicted values can be exported as a python dictionary using the function `sim.PotencyToDict()` or saved in a csv file: `sim.PotencyToCSV()`. 

## Congratulations!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with SSB toolkit, we encourage you to finish the rest of the tutorials in this series. 

## Cite Us

If you use or adapt this tutorial for your own research projects please cite us.

```
@article{ribeiro_ssb_2022,
    title={{SSB} toolkit: from molecular structure to subcellular signaling pathways.},
    author={Ribeiro, Rui Pedro and Gossen, Jonas and Rossetti, Giulia and Giorgetti, Alejandro},
    publisher={bioRxiv},
    url={https://www.biorxiv.org/content/10.1101/2022.11.08.515595v1},
    doi={10.1101/2022.11.08.515595},
    year={2022}
}
```


## Acknowledgments

EU Human Brain Project (SGA1 and SGA2): This open source software was developed in part in the Human Brain Project funded from the European Union's Horizon 2020 Framework Programme for Research and Innovation under Specific Grant Agreements No 720270 and No. 78907 (Human Brain Project SGA1 and SGA2).

<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
    <img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1642677502/logos/COFUNDED_EU_j2ktlp.jpg" width="300" align='left' style="margin-left:50px">
</div>  